## Setup

In [1]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [2]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 

OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]

OUTLIER_ZSCORE = {"STD" : [10.1, 10.2, 10.3, 10.4]}
OUTLIER_KNN = {"K": [3], "X": [1400, 1600]}
OUTLIER_DBSCAN = {"K": [3,4], "X": [500,1500], "EPS": [0.5], "MIN_SAMPLES":  [6.6]}
OUTLIER_NOISE = {"CV": [2.5, 3, 3.5], "DEPTH":[3,5], "MULTI": [5,10]}

FEATURE_SEL_CORR = {"THRESHOLD_CORR": [0.2, 0.25, 0.35]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [3]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [4]:
df = dropMissingValues(df) 

## Outlier Detection

### ZScore

In [5]:
# try:
#     # Outlier Detection
#     outlier_dict_all = {}

#     if OUTLIER99 == True:
#         print("outlier_99")
#         outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
#         df = drop99_all(df, outlier_list_99 )
#         #outlier_dict_all["99_mean"] = outlier_list_99
#         #outlier_dict_all["99_drop"] = outlier_list_99
#         print("done\n")

#     print('starting z_score')
#     for p in OUTLIER_ZSCORE["STD"]:
#         print(p)
#         outlier_list_z_score = z_score(df, p)
#         outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

#     ''''''
#     outlier_lists_all = list(outlier_dict_all.values())
#     outlier_lists_keys = list(outlier_dict_all.keys())
#     combination_list = []
#     for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
#         if not i == 1: #emtpy tuple
#             combination_list.append(combo)
#             # print('combo #{}: {}'.format(i, combo))
#     #print("...")
#     #for c in combination_list[10:]:
#         #print(bold("Combination: "), blue(c),"\n",)
#     #print(bold(len(combination_list)), " total Combinations")

#     model_obj_list = []
#     for i, c in enumerate(combination_list):
#         #print(f'\n{bold(i)}: {blue(c)}')
#         outlier_list = get_unique_list(outlier_dict_all, c)
#         try: 
#             if "99_mean" in c:
#                 X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
#             else:
#                 X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

#             #TODO Regressionmodels
#             model = LinearRegression()
#             obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
#             model_obj_list.append(obj)

#         except Exception as e:
#             print(bold("Error"), red(e))

#     #Summary
#     df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
#     for o in model_obj_list:
#         #print(f'{o.get_type()}, {o.get_comb()}')
#         o.train()
#         mae, score = o.summary()
#         comb = o.get_comb()
#         row = {'combo': comb, 'mae': mae, 'score': score}
#         df_summary = df_summary.append(row, ignore_index=True)

#     df_summary.sort_values("mae").head()

#     ##row
#     mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#     score_best = df_summary[df_summary.score==df_summary.score.max()]
#     best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
#     print(best_model_obj)
#     #print(f'{bold("Lowest mae:")} {mae_best}')

#     model_obj_list_best = [best_model_obj]
#     df_feature_reduction = best_model_obj.get_df().head()

#     # print(df_feature_reduction)

#     ###Feature
#     for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
#         #print(threshold, type(threshold))
#         try: 
#             rel_features = getRelFeatures(df_feature_reduction, threshold)
#             #print(len(rel_features), rel_features)
#             df_feature_reduction = df_feature_reduction[rel_features]
#             X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#             best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
#             # best_model_obj_new.train()
#             model_obj_list_best.append(best_model_obj_new)

#         except Exception as e:
#             print(bold("Error"), red(e))
#     #print(model_obj_list_best)

#     #Summary
#     df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
#     for o in model_obj_list_best:
#         #print(f'{o.get_type()}, {o.get_comb()}')
#         o.train()
#         mae, score = o.summary()
#         comb = o.get_comb()
#         features = o.get_df().columns.to_list()
#         features.remove("price")
#         row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
#         df_summary_best = df_summary_best.append(row, ignore_index=True)

#     #df_summary_best.sort_values("mae")

#     #row
#     mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
#     score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
#     best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
#     print(best_model_obj)
#     print(f'{bold("Lowest mae:")} {mae_best}')

#     lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

#     #import os.path
#     #file_exists = os.path.exists('parameter_search.txt')
#     # if file_exists == False:
#     #     with open('parameter_search.txt', 'w') as f:
#     #         f.write('Create a new text file!')

#     with open('parameter_search.txt', 'a+') as f:
#         for line in lines:
#             f.write(line)
#         f.write('\n')
    
# except Exception as e:
#     print(e)


### KNN

In [6]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")


    print('done \n\nstarting knn')
    for k in OUTLIER_KNN["K"]:
        for o in OUTLIER_KNN["X"]:
            print(k, o)
            outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
            outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction,df_feature_reduction.columns.to_list())
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting knn
3 1400
3 1600
dropped 1400 / 1400 rows
dropped 1600 / 1600 rows
dropped 1600 / 1600 rows
Model: LinearRegression() Combination: ('knn_3_1600',) Features: 25
X has 24 features, but LinearRegression is expecting 11 features as input.


### DB_SCAN

In [7]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")


    print('done \n\nstarting db_scan')
    for k in OUTLIER_DBSCAN["K"]:
        for o in OUTLIER_DBSCAN["X"]:
            for e in OUTLIER_DBSCAN["EPS"]:
                for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                    print(k, o, e, m)
                    outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                    outlier_dict_all["db_scan_"+str(k)+"_"+str(o)+"_"+str(e)+"_"+str(m)] = outlier_list_dbscan


    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting db_scan
3 500 0.5 6.6
3 1500 0.5 6.6
4 500 0.5 6.6
4 1500 0.5 6.6
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
dropped 1397 / 1397 rows
Model: LinearRegression() Combination: ('db_scan_3_500_0.5_6.6',) Features: 25
Error __init__() missing 1 required positional argument: 'features'
Error __init__() missing 1 required positional argument: 'features'
Error __init__() missing 1 required positional argument: 'features'
Model: LinearRegression() Combination: ('db_scan_3_500_0.5_6.6',) Features: 25
Lowest mae:                       combo        mae     score  \
0  (db_scan_3_500_0.5_6.6,)  161024.91  0.468065   

                                            fea

### Noise

In [9]:
try:
    # Outlier Detection
    outlier_dict_all = {}

    if OUTLIER99 == True:
        print("outlier_99")
        outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
        df = drop99_all(df, outlier_list_99 )
        #outlier_dict_all["99_mean"] = outlier_list_99
        #outlier_dict_all["99_drop"] = outlier_list_99
        print("done\n")

    print('done \n\nstarting noise')
    for cv in OUTLIER_NOISE["CV"]:
        for d in OUTLIER_NOISE["DEPTH"]:
            for m in OUTLIER_NOISE["MULTI"]:
                print(cv, d, m)
                outlier_list_noise = getNoise(df, cv, d, m)
                outlier_dict_all["noise_"+str(cv)+"_"+str(d)+"_"+str(m)] = outlier_list_noise

    print('done')


    ''''''
    outlier_lists_all = list(outlier_dict_all.values())
    outlier_lists_keys = list(outlier_dict_all.keys())
    combination_list = []
    for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
        if not i == 1: #emtpy tuple
            combination_list.append(combo)
            # print('combo #{}: {}'.format(i, combo))
    #print("...")
    #for c in combination_list[10:]:
        #print(bold("Combination: "), blue(c),"\n",)
    #print(bold(len(combination_list)), " total Combinations")

    model_obj_list = []
    for i, c in enumerate(combination_list):
        #print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            if "99_mean" in c:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
            else:
                X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

            #TODO Regressionmodels
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))

    #Summary
    df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
    for o in model_obj_list:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        row = {'combo': comb, 'mae': mae, 'score': score}
        df_summary = df_summary.append(row, ignore_index=True)

    df_summary.sort_values("mae").head()

    ##row
    mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
    score_best = df_summary[df_summary.score==df_summary.score.max()]
    best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
    print(best_model_obj)
    #print(f'{bold("Lowest mae:")} {mae_best}')

    model_obj_list_best = [best_model_obj]
    df_feature_reduction = best_model_obj.get_df().head()

    # print(df_feature_reduction)

    ###Feature
    for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
        #print(threshold, type(threshold))
        try: 
            rel_features = getRelFeatures(df_feature_reduction, threshold)
            #print(len(rel_features), rel_features)
            df_feature_reduction = df_feature_reduction[rel_features]
            X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
            best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction, df_feature_reduction.columns.to_list() )
            # best_model_obj_new.train()
            model_obj_list_best.append(best_model_obj_new)

        except Exception as e:
            print(bold("Error"), red(e))
    #print(model_obj_list_best)

    #Summary
    df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
    for o in model_obj_list_best:
        #print(f'{o.get_type()}, {o.get_comb()}')
        o.train()
        mae, score = o.summary()
        comb = o.get_comb()
        features = o.get_df().columns.to_list()
        features.remove("price")
        row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
        df_summary_best = df_summary_best.append(row, ignore_index=True)

    #df_summary_best.sort_values("mae")

    #row
    mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
    score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
    best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
    print(best_model_obj)
    print(f'{bold("Lowest mae:")} {mae_best}')

    lines = f'{df_summary_best["combo"][0]} - {df_summary_best["mae"][0]} - {len(df_summary_best["features"][0])} ' 

    #import os.path
    #file_exists = os.path.exists('parameter_search.txt')
    # if file_exists == False:
    #     with open('parameter_search.txt', 'w') as f:
    #         f.write('Create a new text file!')

    with open('parameter_search.txt', 'a+') as f:
        for line in lines:
            f.write(line)
        f.write('\n')
    
except Exception as e:
    print(e)


outlier_99
done

done 

starting noise
2.5 3 5
Expected cv as an integer, cross-validation object (from sklearn.model_selection) or an iterable. Got 2.5.
